In [1]:
%%sh

# The name of our algorithm
algorithm_name=image-classifier

cd container

chmod +x code/train-mnist
#chmod +x code/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

# On a SageMaker Notebook Instance, the docker daemon may need to be restarted in order
# to detect your network configuration correctly.  (This is a known issue.)
if [ -d "/home/ec2-user/SageMaker" ]; then
  sudo service docker restart
fi

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Stopping docker: [  OK  ]
Starting docker:	.[  OK  ]
Sending build context to Docker daemon  24.58kB
Step 1/12 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.0.0-cpu-py36-ubuntu18.04
 ---> a709409e7f8c
Step 2/12 : RUN apt-get update &&     apt-get install -y nginx
 ---> Using cache
 ---> 031ff00c7cd9
Step 3/12 : RUN pip install gevent gunicorn flask
 ---> Using cache
 ---> fe4bd0c9aa1f
Step 4/12 : ENV PATH="/opt/program:${PATH}"
 ---> Using cache
 ---> 226cacf9a480
Step 5/12 : COPY code /opt/program
 ---> 0a111ac59797
Step 6/12 : WORKDIR /opt/program
 ---> Running in a3c87d13f2a0
Removing intermediate container a3c87d13f2a0
 ---> 518f35bb47c6
Step 7/12 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.0.0-cpu-py36-ubuntu18.04
 ---> a709409e7f8c
Step 8/12 : RUN apt-get update &&     apt-get install -y nginx
 ---> Using cache
 ---> 031ff00c7cd9
Step 9/12 : RUN pip install gevent gunicorn flask
 ---> Using cache
 ---> fe4b

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [2]:
# S3 prefix
prefix = 'image-classifier'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [3]:
# Create session
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [4]:
print(sess.default_bucket())

sagemaker-us-east-1-731833107751


In [5]:
# upload data for training
WORK_DIRECTORY = 'mnist-data' 
 
bucket = sess.default_bucket()

s3 = boto3.client('s3')
s3.download_file(bucket, 'mnist-data/x_train.npy','data/x_train.npy')
s3.download_file(bucket, 'mnist-data/y_train.npy','data/y_train.npy')
s3.download_file(bucket, 'mnist-data/x_test.npy','data/x_test.npy')
s3.download_file(bucket, 'mnist-data/y_test.npy','data/y_test.npy')
 
  

In [6]:
data_location = "s3://{}/mnist-data".format(sess.default_bucket()) #"s3://" + sess.default_bucket() + "/mnist-data"  
print(data_location)
print(sess.default_bucket())

s3://sagemaker-us-east-1-731833107751/mnist-data
sagemaker-us-east-1-731833107751


In [12]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/image-classifier:latest'.format(account, region)

from sagemaker.tensorflow import TensorFlow

estimator = sage.estimator.Estimator(image,
                       role=role, 
                       train_instance_count=1, 
                       train_instance_type='local',
                       #train_instance_type='ml.c5.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),                       
                       sagemaker_session=sess)

# estimator = TensorFlow(image=image,
#                              role=role,
#                              train_instance_count=1,
#                              train_instance_type='local',
#                              #train_instance_type='ml.m5.4xlarge',
#                              output_path="s3://{}/output".format(sess.default_bucket()),
#                              framework_version='2.0.0',
#                              py_version='py3',
#                              sagemaker_session=sess,
#                              distributions={'parameter_server': {'enabled': True}})

estimator.fit(data_location)

2020-02-06 22:22:22 Starting - Starting the training job...
2020-02-06 22:22:25 Starting - Launching requested ML instances......
2020-02-06 22:23:28 Starting - Preparing the instances for training...
2020-02-06 22:24:23 Downloading - Downloading input data
2020-02-06 22:24:23 Training - Downloading the training image......
2020-02-06 22:25:18 Uploading - Uploading generated training model
2020-02-06 22:25:18 Failed - Training job failed
2020-02-06 22:25:13,535 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-02-06 22:25:13,541 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-02-06 22:25:13,552 sagemaker-containers ERROR    Reporting training FAILURE
2020-02-06 22:25:13,552 sagemaker-containers ERROR    framework error: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/sagemaker_containers/_trainer.py", line 81, in train
    entrypoint()
  File "/usr/local/lib/python3.6/dist-p

UnexpectedStatusException: Error for Training job image-classifier-2020-02-06-22-22-22-554: Failed. Reason: AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/sagemaker_containers/_trainer.py", line 81, in train
    entrypoint()
  File "/usr/local/lib/python3.6/dist-packages/sagemaker_tensorflow_container/training.py", line 211, in main
    train(env, framework.mapping.to_cmd_args(user_hyperparameters))
  File "/usr/local/lib/python3.6/dist-packages/sagemaker_tensorflow_container/training.py", line 158, in train
    runner=runner_type)
  File "/usr/local/lib/python3.6/dist-packages/sagemaker_containers/entry_point.py", line 93, in run
    install(user_entry_point, _env.code_dir, capture_error)
  File "/usr/local/lib/python3.6/dist-packages/sagemaker_containers/entry_point.py", line 118, in install
    entrypoint_type = _entry_point_type.get(dst, name)
  File "/usr/local/lib/python3.6/dist-packages/sagemaker_containers/_en